In [158]:
import numpy as np
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline
import os
import pymysql
import pandas as pd
import sys
import datetime

import pickle

In [90]:
# Declare and initialize database connection credentials
URL="database-1.cyhnb62nmtav.eu-west-1.rds.amazonaws.com"
PASSWORD=11223344
PORT="3306"
USER ="kuroko"
DB = 'dbbikes'

# Attempt connection to database
# Print a statement on the screen to check whether the connection is working
try:
    engine = create_engine("mysql+pymysql://{}:{}@{}:{}".format(USER,PASSWORD,URL,PORT),echo=True)
    print('+=========================+')
    print('|  CONNECTED TO DATABASE  |')
    print('+=========================+')
    engine.execute("use dbbikes;")
    
# Exit if connection not working   
except Exception as e:
        sys.exit(e)

+=========================+
|  CONNECTED TO DATABASE  |
+=========================+
2022-04-06 19:00:25,558 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-04-06 19:00:25,559 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-06 19:00:25,564 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-04-06 19:00:25,565 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-06 19:00:25,576 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-04-06 19:00:25,577 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-06 19:00:25,591 INFO sqlalchemy.engine.Engine use dbbikes;
2022-04-06 19:00:25,592 INFO sqlalchemy.engine.Engine [raw sql] {}


In [91]:
# Create dataframe and store data running SQL query
rows = engine.execute("SELECT * from dbbikes.availability;")
df = []
for row in rows:
        df.append(dict(row))
df=pd.DataFrame(df)
# Examine dataframe object, show first 10 rows
df.head(10)

2022-04-06 19:00:25,652 INFO sqlalchemy.engine.Engine SELECT * from dbbikes.availability;
2022-04-06 19:00:25,653 INFO sqlalchemy.engine.Engine [raw sql] {}


,number,available_bikes,available_bike_stands,last_update
0,42,30,0,1646881143000
1,30,11,9,1646881067000
2,54,0,33,1646881296000
3,108,8,27,1646881295000
4,56,13,27,1646881294000
5,6,10,10,1646881235000
6,18,12,18,1646881296000
7,32,7,23,1646881290000
8,52,0,32,1646881288000
9,48,24,16,1646881287000


In [92]:
type(df['last_update'])

pandas.core.series.Series

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456409 entries, 0 to 456408
Data columns (total 4 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   number                 456409 non-null  object
 1   available_bikes        456409 non-null  object
 2   available_bike_stands  456409 non-null  object
 3   last_update            456409 non-null  object
dtypes: object(4)
memory usage: 13.9+ MB


In [94]:
df['number'] = df['number'].astype(int)

In [95]:
df['available_bikes'] = df['available_bikes'].astype(int)

In [96]:
df['available_bike_stands'] = df['available_bike_stands'].astype(int)

In [97]:

df['last_update'] = df['last_update'].astype(int)

In [98]:
df['last_update'] = df['last_update']//1000

In [99]:
df['last_update'] = df['last_update'].astype(str)
df['last_update']

0         1646881143
1         1646881067
2         1646881296
3         1646881295
4         1646881294
             ...    
456404    1649267510
456405    1649267401
456406    1649267522
456407    1649267507
456408    1649267437
Name: last_update, Length: 456409, dtype: object

In [106]:
df['last_update'] = pd.to_datetime(df['last_update'], unit='s')

In [107]:
df['last_update']

0        2022-03-10 02:59:03
1        2022-03-10 02:57:47
2        2022-03-10 03:01:36
3        2022-03-10 03:01:35
4        2022-03-10 03:01:34
                 ...        
456404   2022-04-06 17:51:50
456405   2022-04-06 17:50:01
456406   2022-04-06 17:52:02
456407   2022-04-06 17:51:47
456408   2022-04-06 17:50:37
Name: last_update, Length: 456409, dtype: datetime64[ns]

In [156]:
df['day_of_week'] = df['last_update'].dt.dayofweek + 1

In [119]:
df['hour'] = df['last_update'].dt.hour

In [129]:
df['date'] = df['last_update'].dt.date

In [120]:
df['hour'].unique()

array([ 2,  3, 10,  9,  1, 16, 17, 18, 19, 20, 21, 22, 23,  0,  4,  5,  6,
        7,  8, 11, 12, 13, 14, 15])

In [130]:
df.tail(10)

,number,available_bikes,available_bike_stands,last_update,day_of_week,hour,date
456399,3,4,16,2022-04-06 17:45:30,3,17,2022-04-06
456400,40,15,6,2022-04-06 17:51:33,3,17,2022-04-06
456401,29,23,6,2022-04-06 17:47:59,3,17,2022-04-06
456402,103,3,37,2022-04-06 17:46:33,3,17,2022-04-06
456403,28,17,13,2022-04-06 17:50:54,3,17,2022-04-06
456404,39,6,14,2022-04-06 17:51:50,3,17,2022-04-06
456405,83,7,33,2022-04-06 17:50:01,3,17,2022-04-06
456406,92,23,17,2022-04-06 17:52:02,3,17,2022-04-06
456407,21,13,17,2022-04-06 17:51:47,3,17,2022-04-06
456408,88,3,27,2022-04-06 17:50:37,3,17,2022-04-06


In [122]:
df.dtypes

number                            int64
available_bikes                   int64
available_bike_stands             int64
last_update              datetime64[ns]
day_of_week                       int64
hour                              int64
dtype: object

In [124]:
# Create dataframe and store data running SQL query
rows = engine.execute("SELECT * from dbbikes.weather;")
df2 = []
for row in rows:
        df2.append(dict(row))
df2=pd.DataFrame(df2)
# Examine dataframe object, show first 10 rows
df2.head(10)

2022-04-06 20:09:50,768 INFO sqlalchemy.engine.Engine SELECT * from dbbikes.weather;
2022-04-06 20:09:50,771 INFO sqlalchemy.engine.Engine [raw sql] {}


,dt,description,icon,temperture,pressure,humidity,visibility
0,1647201841,scattered clouds,03n,281.08,1006,74,10000
1,1647023435,broken clouds,04n,282.22,993,84,10000
2,1647025107,broken clouds,04n,281.92,993,87,10000
3,1647201841,scattered clouds,03n,281.08,1006,74,10000
4,1647201041,scattered clouds,03n,280.94,1005,74,10000
5,1647181434,broken clouds,04d,284.58,997,70,10000
6,1647201841,scattered clouds,03n,281.08,1006,74,10000
7,1647201041,scattered clouds,03n,280.94,1005,74,10000
8,1647028909,broken clouds,04n,280.94,994,90,10000
9,1647025521,broken clouds,04n,281.72,994,88,10000


In [125]:
df2['dt'] = pd.to_datetime(df2['dt'], unit='s')

In [126]:
df2['dt']

<bound method DataFrame.info of                      dt       description icon temperture pressure humidity  \
0   2022-03-13 20:04:01  scattered clouds  03n     281.08     1006       74   
1   2022-03-11 18:30:35     broken clouds  04n     282.22      993       84   
2   2022-03-11 18:58:27     broken clouds  04n     281.92      993       87   
3   2022-03-13 20:04:01  scattered clouds  03n     281.08     1006       74   
4   2022-03-13 19:50:41  scattered clouds  03n     280.94     1005       74   
..                  ...               ...  ...        ...      ...      ...   
578 2022-04-06 15:09:40  scattered clouds  03d     285.21      985       66   
579 2022-04-06 16:09:40  scattered clouds  03d     282.81      985       78   
580 2022-04-06 17:09:41  scattered clouds  03d     281.57      984       85   
581 2022-04-06 18:06:46     broken clouds  04d      281.4      983       81   
582 2022-04-06 19:09:41  scattered clouds  03n     281.13      982       75   

    visibility  
0 

In [135]:
df2['date'] = df2['dt'].dt.date
df2['hour'] = df2['dt'].dt.hour

In [136]:
# Merge dataframes
mergedStuff = pd.merge(df, df2, on = ['date', 'hour'], how = 'inner')
# Show first 2 scraps (oldest) for stations (113*2=226)
mergedStuff.head(226)

,number,available_bikes,available_bike_stands,last_update,day_of_week,hour,date,dt,description,icon,temperture,pressure,humidity,visibility
0,54,0,33,2022-03-10 03:01:36,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
1,108,8,27,2022-03-10 03:01:35,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
2,56,13,27,2022-03-10 03:01:34,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
3,6,10,10,2022-03-10 03:00:35,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
4,18,12,18,2022-03-10 03:01:36,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,11,14,16,2022-03-10 03:01:26,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
222,17,1,19,2022-03-10 03:01:30,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
223,45,21,9,2022-03-10 03:09:41,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000
224,114,1,39,2022-03-10 03:01:36,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,277.69,1008,96,6000


In [144]:
mergedStuff['visibility'].unique()

array(['6000', '10000', '9000', '8000', '5000', '900', '300', '800',
       '2600', '4300', '1500', '600', '500', '2000', '150', '200', '700',
       '3000', '4000', '7000', '4500', '3500'], dtype=object)

In [140]:
# Create a separate dataframe with days of the week (categorical)
data_input1 = pd.DataFrame(mergedStuff['day_of_week'])

# Create a separate dataframe with cloud coverage information (categorical)
data_input2 = pd.DataFrame(mergedStuff['description'])

# Concatenate the two dataframes in the main one
dummy = pd.get_dummies(data_input1)
dummy_2 = pd.get_dummies(data_input2)
mergedStuff = pd.concat([mergedStuff,dummy],axis=1)
mergedStuff = pd.concat([mergedStuff,dummy_2],axis=1)

In [149]:
mergedStuff.columns

Index(['number', 'available_bikes', 'available_bike_stands', 'last_update',
       'day_of_week', 'hour', 'date', 'dt', 'description', 'icon',
       'temperture', 'pressure', 'humidity', 'visibility', 'day_of_week',
       'description_broken clouds', 'description_clear sky',
       'description_few clouds', 'description_fog', 'description_haze',
       'description_light intensity drizzle rain',
       'description_light intensity shower rain', 'description_light rain',
       'description_mist', 'description_moderate rain',
       'description_overcast clouds', 'description_scattered clouds'],
      dtype='object')

In [154]:
mergedStuff.head(10)

,number,available_bikes,available_bike_stands,last_update,day_of_week,hour,date,dt,description,icon,...,description_few clouds,description_fog,description_haze,description_light intensity drizzle rain,description_light intensity shower rain,description_light rain,description_mist,description_moderate rain,description_overcast clouds,description_scattered clouds
0,54,0,33,2022-03-10 03:01:36,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
1,108,8,27,2022-03-10 03:01:35,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
2,56,13,27,2022-03-10 03:01:34,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
3,6,10,10,2022-03-10 03:00:35,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
4,18,12,18,2022-03-10 03:01:36,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
5,32,7,23,2022-03-10 03:01:30,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
6,52,0,32,2022-03-10 03:01:28,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
7,48,24,16,2022-03-10 03:01:27,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
8,13,5,25,2022-03-10 03:01:23,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0
9,43,29,1,2022-03-10 03:01:36,4,3,2022-03-10,2022-03-10 03:51:01,broken clouds,04n,...,0,0,0,0,0,0,0,0,0,0


### Predict the number of available bikes

In [150]:
# Select model features and store them in a new dataframe
input_model = pd.DataFrame(mergedStuff[['number', 'hour', 'temperture', 
'visibility', 'day_of_week','description_broken clouds', 'description_clear sky',
       'description_few clouds', 'description_fog', 'description_haze',
       'description_light intensity drizzle rain',
       'description_light intensity shower rain', 'description_light rain',
       'description_mist', 'description_moderate rain',
       'description_overcast clouds', 'description_scattered clouds']])

# Define target variable
output = mergedStuff['available_bikes']

# Split dataset to train and test
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.33,random_state=42)
print("Training the model on %s rows and %s columns." % X_train.shape)

# Instantiate RandomForestRegressor object calling 100 decision tree models
rf = RandomForestRegressor(n_estimators=100)

# Train the model
rf.fit(X_train, Y_train)

print("Testing the model on %s rows." % Y_test.shape[0])


Training the model on 305240 rows and 18 columns.
Testing the model on 150343 rows.


In [151]:
LinearReg=LinearRegression()
LinearReg.fit(X_train,Y_train)
pred=LinearReg.predict(X_test)
pred1=pd.DataFrame(pred)
print (pred1)
print(X_test)
print(np.array(X_test))

                0
0       13.055206
1       13.090938
2       12.279806
3       12.885819
4       13.120419
...           ...
150338  12.269845
150339  12.274757
150340  12.683870
150341  12.343575
150342  12.320555

[150343 rows x 1 columns]
        number  hour temperture visibility  day_of_week  day_of_week  \
88914      116     3     282.17      10000            6            6   
36193      103    10     279.79      10000            3            3   
205456      28     2     281.11      10000            5            5   
42613      115    19     280.25      10000            3            3   
248375      98     6      279.4      10000            7            7   
...        ...   ...        ...        ...          ...          ...   
102419      75    20     284.52      10000            6            6   
26828       17     0     282.26      10000            3            3   
107027      24     2     280.94      10000            7            7   
202566      22    22     282.35      

In [152]:
# Get prediction for test cases
prediction = rf.predict(X_test)

# Show the predicted test cases
print(prediction)
print(Y_test)

[ 9.         10.02138889 12.         ... 16.         18.66247888
  1.63865007]
88914      9
36193     11
205456    12
42613      7
248375    38
          ..
102419     2
26828      1
107027    16
202566    18
300338     3
Name: available_bikes, Length: 150343, dtype: int64


In [153]:
print("RMSE: %f" % np.sqrt(metrics.mean_squared_error(Y_test,prediction)))

RMSE: 1.500227


In [157]:
df2['description'].unique()

array(['scattered clouds', 'broken clouds', 'few clouds', 'clear sky',
       'overcast clouds', 'light rain', 'moderate rain', 'mist', 'fog',
       'haze', 'light intensity shower rain',
       'light intensity drizzle rain'], dtype=object)

In [162]:
pickle.dump(rf, open('final_prediction.pickle', 'wb'))

In [163]:
# This is not strictly functional to the application
random_forest = pickle.load(open("final_prediction.pickle", "rb"))

### Predict the number of available bike stands

In [165]:
# Select model features and store them in a new dataframe
input_model = pd.DataFrame(mergedStuff[['number', 'hour', 'temperture', 
'visibility', 'day_of_week','description_broken clouds', 'description_clear sky',
       'description_few clouds', 'description_fog', 'description_haze',
       'description_light intensity drizzle rain',
       'description_light intensity shower rain', 'description_light rain',
       'description_mist', 'description_moderate rain',
       'description_overcast clouds', 'description_scattered clouds']])

# Define target variable
output = mergedStuff['available_bike_stands']

# Split dataset to train and test
X_train_2,X_test_2,Y_train_2,Y_test_2=train_test_split(input_model,output,test_size=0.33,random_state=42)
print("Training the model on %s rows and %s columns." % X_train_2.shape)

# Instantiate RandomForestRegressor object calling 100 decision tree models
rf2 = RandomForestRegressor(n_estimators=100)

# Train the model
rf2.fit(X_train_2, Y_train_2)

print("Testing the model on %s rows." % Y_test_2.shape[0])


Training the model on 305240 rows and 18 columns.
Testing the model on 150343 rows.


In [166]:
# Get prediction for test cases
prediction_bike_stands = rf2.predict(X_test_2)
# Show the predicted test cases
print (prediction_bike_stands)
print(Y_test_2)

[21.         29.99379762 18.01       ...  4.          1.35719661
 28.37252067]
88914     21
36193     29
205456    18
42613     23
248375     2
          ..
102419    38
26828     19
107027     4
202566     2
300338    27
Name: available_bike_stands, Length: 150343, dtype: int64


In [167]:
print("RMSE: %f" % np.sqrt(metrics.mean_squared_error(Y_test_2,prediction_bike_stands)))


RMSE: 1.519969


In [169]:
import pickle

pickle.dump(rf2,open('final_prediction_bike_stands.pickle', 'wb'))

In [170]:
# This is not strictly functional to the application
random_forest_stands=pickle.load(open("final_prediction_bike_stands.pickle", "rb"))

In [ ]:
df2.info